In [378]:
import pandas as pd

# Exploring

In [379]:
customer_df = pd.read_csv('customers.csv')
customer_df.head(10)

,id,firstname,lastname
0,0,John,Maxwell
1,1,John,Heisenberg
2,2,John,Fermi
3,3,John,Feynman
4,4,John,Lavoisier
5,5,John,Pasteur
6,6,James,Maxwell
7,7,James,Heisenberg
8,8,James,Fermi
9,9,James,Feynman


In [380]:
orders_df = pd.read_csv('orders.csv')
orders_df.head(10)

,id,customer,products
0,0,0,1 0 1 0
1,1,22,0 5 0 4 5 3 2 1 1
2,2,57,5 2 4
3,3,20,4 3 0 2 4
4,4,28,5 0 5 0 3 4
5,5,40,0 1 0 2
6,6,32,5 5 0 5 1 5 1 3 1
7,7,5,5 0 3 2
8,8,45,1 2 1 0
9,9,37,0 5 5 5 2


In [381]:
products_df = pd.read_csv('products.csv')
products_df.head(10)

,id,name,cost
0,0,screwdriver,2.981164
1,1,wrench,6.490396
2,2,hammer,2.903732
3,3,sickle,8.901570
4,4,candle,9.806495
5,5,bandsaw,10.435252


# Sanitization

In [382]:
sanitized_orders_df = orders_df.copy()
sanitized_orders_df['products'] = sanitized_orders_df['products'].str.split(' ')
sanitized_orders_df = sanitized_orders_df.explode('products').astype('int64')
sanitized_orders_df.head(5)

,id,customer,products
0,0,0,1
0,0,0,0
0,0,0,1
0,0,0,0
1,1,22,0


In [383]:
orders_prices_df = sanitized_orders_df.merge(products_df,left_on='products',right_on="id",suffixes=(None,"_product"))
orders_prices_df.head(5)

,id,customer,products,id_product,name,cost
0,0,0,1,1,wrench,6.490396
1,0,0,1,1,wrench,6.490396
2,1,22,1,1,wrench,6.490396
3,1,22,1,1,wrench,6.490396
4,5,40,1,1,wrench,6.490396


-----

# Task 1

In [384]:
task1_df = orders_prices_df.groupby(['id'])['cost'].sum()
task1_df = task1_df.reset_index().rename(columns={"cost": "euros"})
task1_df.head(5)

,id,euros
0,0,18.943120
1,1,61.425421
2,2,23.145479
3,3,34.399455
4,4,45.540896


In [385]:
task1_df.to_csv('order_prices.csv', index=False)

# Task 2

In [386]:
task2_df = orders_prices_df.copy()
task2_df['customer'] = task2_df['customer'].astype('string')

In [387]:
task2_df = task2_df.groupby('products').customer.apply(set).reset_index()
task2_df['customer'] = task2_df['customer'].apply(lambda x: ' '.join(x))
task2_df = task2_df.rename(columns={"products": "id", "customer": "customer_ids"})

task2_df.head(5)

,id,customer_ids
0,0,20 50 59 6 21 24 45 46 17 32 28 54 40 0 34 44 ...
1,1,51 50 6 24 45 46 17 32 41 58 9 3 40 0 34 35 44...
2,2,20 51 57 6 21 24 45 46 17 41 9 54 40 44 37 48 ...
3,3,20 51 50 21 24 46 17 32 41 9 28 54 34 44 38 47...
4,4,20 51 59 57 21 24 17 41 32 9 36 28 34 35 44 10...


In [388]:
task2_df.to_csv('product_customers.csv', index=False)

# Task 3

In [389]:
task3_df = orders_prices_df.groupby(['customer'])['cost'].sum().reset_index()
task3_df = customer_df.merge(task3_df, left_on='id',right_on="customer")
task3_df = task3_df.drop(columns='customer').rename(columns={"cost": "total_euros"})
task3_df = task3_df.sort_values(by = 'total_euros', ascending=False)
task3_df.head(5)

,id,firstname,lastname,total_euros
18,34,Samuel,Lavoisier,145.254336
25,44,Mary,Fermi,138.410627
13,24,Ringo,Maxwell,136.827670
12,22,Paul,Lavoisier,124.710690
17,32,Samuel,Fermi,119.163897


In [390]:
task3_df.to_csv('customer_ranking.csv', index=False)